# Results p. 2
## Fixation Temporal Alignment
### Comparing RTO & RTD between Detectors

In [18]:
import os

import cv2
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from docutils.nodes import legend
from numpy.ma.extras import vstack
from plotly.subplots import make_subplots
import plotly.io as pio

import peyes

import analysis.utils as u
from analysis._article_results.lund2013._helpers import *
import analysis.statistics.channel_time_diffs as ctd
import analysis.statistics.channel_sdt as ch_sdt

# pio.renderers.default = "browser"
FIG_ID, PANEL_ID, IS_SUPP = 4, 'A', False
EVENT_TYPE = "Fixation"
W, H = 600, 400

In [19]:
LABEL = 1        # EventLabelEnum.FIXATION.value
MATCHING_THRESHOLD = 20     # samples; matches with time-diff greater than this value are not considered "hits"

## Load Data
The `time_diffs` table describes the difference in onset/offset timings between the ground truth and the predicted fixations.  
The `thresholded_time_diffs` table is a subset of `time_diffs` where only the time-diffs whose absolute value is less than or equal to the `MATCHING_THRESHOLD` are kept.

In [20]:
time_diffs = ctd.load(
    DATASET_NAME,
    PROCESSED_DATA_DIR,
    label=LABEL,
    stimulus_type=STIMULUS_TYPE,
)

thresholded_time_diffs = time_diffs.map(lambda vals: np.fromiter(filter(lambda v: abs(v) <= MATCHING_THRESHOLD, vals), dtype=int))

thresholded_time_diffs.T

channel_type                                                      onset  \
trial_id gt pred                                                          
25       RA ivt       [1, -11, -7, -11, -3, -2, -3, -3, -1, -11, -5,...   
            idt       [0, -17, -18, -16, -14, -17, -17, -9, -19, -15...   
            engbert   [3, -8, -1, -1, -2, -1, -2, -2, -1, 0, -3, 0, ...   
            nh        [0, -1, 1, -2, 4, 2, 5, 7, 2, 7, 1, -9, -8, -3...   
            ivvt      [1, -7, -1, -2, 5, 0, -2, -1, 0, 0, -2, -1, 4,...   
...                                                                 ...   
44       MN nh        [3, 4, -1, -4, -5, -6, -6, -7, -4, -8, 2, -6, ...   
            ivvt      [-2, -2, -1, -4, -2, -5, -7, -6, -7, -2, -9, -...   
            RA        [0, 0, 0, 0, -3, 0, 0, 0, 1, 1, 0, 0, -1, -2, ...   
            remodnav                   [-1, -1, -4, 0, -5, -5, -10, -4]   
            idvt      [-8, -11, -14, -11, -16, -13, -18, -13, -14, -...   

channel_type                                                     offset  
trial_id gt pred                                                         
25       RA ivt       [1, 2, 2, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 2, 1, ...  
            idt       [4, 3, 1, 3, 3, 3, 3, 4, 4, 4, 1, 6, 4, 4, 5, ...  
            engbert   [-1, 1, 0, -1, -1, -1, -1, -1, -1, -2, -1, -2,...  
            nh        [-6, -3, -4, -6, -8, -5, -6, -3, -10, -5, -4, ...  
            ivvt      [1, 2, 2, 1, 0, 1, 1, 1, 1, 0, 1, 5, 1, 1, 2, ...  
...                                                                 ...  
44       MN nh        [-2, -3, -6, -4, -5, -2, -8, -2, -7, -3, -5, -...  
            ivvt      [1, 1, 1, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1, 2, ...  
            RA        [0, 0, 0, 0, -1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0,...  
            remodnav                            [-1, -2, -2, -1, 0, -3]  
            idvt          [2, 2, 1, 4, 2, 2, 1, 2, 3, 0, 2, 2, 1, 1, 0]  

[266 rows x 2 columns]

## Compute Metrics

_Comments on the calculations:_  
(1) `num_gt_events` is the number of onsets/offsets across all trials, for the human annotator of every _(human, algorithm)_ pair.  
(2) `num_pred_events` is the number of onsets/offsets across all trials, for the algorithm of every _(human, algorithm)_ pair (or the _other human_ annotator).  
  
(3) `num_matches` is the number of matching onsets/offsets between the ground truth and the predicted events.  
  
(4) `hit_rate` (_recall_) is the percentage of matched GT onsets/offsets out of all GTs.  
(5) `ppv` (_precision_) is the percentage of matched PRED onsets/offsets out of all PREDs.  
(6) `f1` is the harmonic mean of `hit_rate` and `ppv`.
  
(7) `RTO` is the mean difference in onset/offset timings (measured in samples).  
(8) `RTD` is the standard deviation of difference in onset/offset timings (measured in samples).  

In [21]:
def calculate_alignment_metrics(td: pd.DataFrame) -> pd.DataFrame:
    num_gt_events, num_pred_events = _calc_num_events()
    aggregate_td = td.unstack(0).unstack(0).apply(np.hstack, axis=1).apply(lambda vals: vals[~np.isnan(vals)]).rename("time_diffs")
    aggregate_td = aggregate_td.reindex(u.sort_labelers(aggregate_td.index.get_level_values(u.PRED_STR)), level=u.PRED_STR)
    num_matches = aggregate_td.apply(len).rename('num_matches')     # number of matching onsets/offsets between the ground truth and the predicted events.
    
    hit_rate = (num_matches / num_gt_events).rename('hit_rate')
    ppv = (num_matches / num_pred_events).rename('ppv')
    f1 = (2 * (hit_rate * ppv) / (hit_rate + ppv)).rename('f1')
    rto = aggregate_td.apply(np.mean).rename('RTO')
    rtd = aggregate_td.apply(np.std).rename('RTD')
    
    alignment = pd.concat([num_gt_events, num_pred_events, num_matches, hit_rate, ppv, f1, rto, rtd], axis=1).unstack(
        [u.GT_STR, u.PRED_STR]).stack(0, future_stack=True)
    alignment.reindex(columns=u.sort_labelers(alignment.columns.get_level_values(u.PRED_STR).unique()), level=1)
    alignment.index.names = [peyes.constants.CHANNEL_TYPE_STR, peyes.constants.METRIC_STR]
    alignment.columns.names = [u.GT_STR, u.PRED_STR]
    return alignment


def _calc_num_events() -> (pd.Series, pd.Series):
    # Loads the channel-SDT data and extracts the number of GT and PRED events (onsets/offsets) for each labeler-detector pair.
    num_events = ch_sdt.load(
        dataset_name=DATASET_NAME,
        output_dir=PROCESSED_DATA_DIR,
        label=LABEL,
        stimulus_type=STIMULUS_TYPE,
        channel_type=None,
    )
    num_events = num_events.loc[(slice(None), ['P', 'PP'], 0)]      # keep only the positive counts
    num_events = num_events.droplevel(peyes.constants.THRESHOLD_STR)
    num_events = num_events.stack([u.GT_STR, u.PRED_STR], future_stack=True).sum(axis=1)
    num_events = num_events.reorder_levels([u.GT_STR, u.PRED_STR, peyes.constants.METRIC_STR, peyes.constants.CHANNEL_TYPE_STR]).sort_index()
    
    num_gt_events = num_events.xs('P', level=peyes.constants.METRIC_STR).rename('num_gt')   # number of onsets/offsets across all trials, for the human annotator of every _(human, algorithm)_ pair.
    num_pred_events = num_events.xs('PP', level=peyes.constants.METRIC_STR).rename('num_pred')  # number of onsets/offsets across all trials, for the algorithm of every _(human, algorithm)_ pair (or the _other human_ annotator).
    return num_gt_events, num_pred_events

In [22]:
alignment_metrics = calculate_alignment_metrics(thresholded_time_diffs)
alignment_metrics

gt                                MN                                      \
pred                              RA     engbert         idt        idvt   
channel_type metric                                                        
offset       num_gt       404.000000  404.000000  404.000000  404.000000   
             num_pred     391.000000  539.000000  257.000000  247.000000   
             num_matches  384.000000  401.000000  226.000000  226.000000   
             hit_rate       0.950495    0.992574    0.559406    0.559406   
             ppv            0.982097    0.743970    0.879377    0.914980   
             f1             0.966038    0.850477    0.683812    0.694316   
             RTO            0.005208   -0.698254    3.013274    3.013274   
             RTD            1.421551    1.463171    3.185254    3.185254   
onset        num_gt       404.000000  404.000000  404.000000  404.000000   
             num_pred     391.000000  539.000000  257.000000  247.000000   
             num_matches  375.000000  400.000000  175.000000  174.000000   
             hit_rate       0.928218    0.990099    0.433168    0.430693   
             ppv            0.959079    0.742115    0.680934    0.704453   
             f1             0.943396    0.848356    0.529501    0.534562   
             RTO            0.917333   -1.315000  -12.902857  -13.040230   
             RTD            4.023411    4.068264    6.345009    6.098199   

gt                                                                        \
pred                             ivt        ivvt          nh    remodnav   
channel_type metric                                                        
offset       num_gt       404.000000  404.000000  404.000000  404.000000   
             num_pred     502.000000  729.000000  327.000000  171.000000   
             num_matches  385.000000  397.000000  307.000000   77.000000   
             hit_rate       0.952970    0.982673    0.759901    0.190594   
             ppv            0.766932    0.544582    0.938838    0.450292   
             f1             0.849890    0.700794    0.839945    0.267826   
             RTO            1.241558    0.342569   -4.960912   -1.987013   
             RTD            1.368035    1.639723    3.751886    4.075564   
onset        num_gt       404.000000  404.000000  404.000000  404.000000   
             num_pred     502.000000  729.000000  327.000000  171.000000   
             num_matches  380.000000  395.000000  293.000000  116.000000   
             hit_rate       0.940594    0.977723    0.725248    0.287129   
             ppv            0.756972    0.541838    0.896024    0.678363   
             f1             0.838852    0.697264    0.801642    0.403478   
             RTO           -3.776316   -0.126582   -0.177474    2.077586   
             RTD            4.779888    4.277013    5.795953    4.191845   

gt                                RA                                      \
pred                              MN     engbert         idt        idvt   
channel_type metric                                                        
offset       num_gt       391.000000  563.000000  563.000000  563.000000   
             num_pred     404.000000  749.000000  358.000000  347.000000   
             num_matches  384.000000  557.000000  313.000000  313.000000   
             hit_rate       0.982097    0.989343    0.555950    0.555950   
             ppv            0.950495    0.743658    0.874302    0.902017   
             f1             0.966038    0.849085    0.679696    0.687912   
             RTO           -0.005208   -0.775583    2.958466    2.958466   
             RTD            1.421551    1.426190    3.661587    3.661587   
onset        num_gt       391.000000  563.000000  563.000000  563.000000   
             num_pred     404.000000  749.000000  358.000000  347.000000   
             num_matches  375.000000  552.000000  221.000000  221.000000   
             hit_rate       0.959079    0.9804

#### Annotator RA

In [23]:
ra_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_onset_alignments = ra_onset_alignments.reindex(
    columns=u.sort_labelers(ra_onset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_onset_alignments

pred,MN,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,391.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000
num_pred,404.000000,696.000000,1042.000000,358.000000,347.000000,749.000000,481.000000,263.000000
num_matches,375.000000,519.000000,549.000000,221.000000,221.000000,552.000000,422.000000,173.000000
hit_rate,0.959079,0.921847,0.975133,0.392540,0.392540,0.980462,0.749556,0.307282
ppv,0.928218,0.745690,0.526871,0.617318,0.636888,0.736983,0.877339,0.657795
f1,0.943396,0.824464,0.684112,0.479913,0.485714,0.841463,0.808429,0.418886
RTO,-0.917333,-4.019268,-0.400729,-12.058824,-12.058824,-1.443841,-0.251185,1.543353
RTD,4.023411,4.421398,4.190335,5.842672,5.842672,3.735906,5.854753,4.269075


In [24]:
ra_offset_alignments = alignment_metrics.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('RA', level=u.GT_STR).dropna(axis=1, how='all')
ra_offset_alignments = ra_offset_alignments.reindex(
    columns=u.sort_labelers(ra_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

ra_offset_alignments

pred,MN,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,391.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000
num_pred,404.000000,696.000000,1042.000000,358.000000,347.000000,749.000000,481.000000,263.000000
num_matches,384.000000,528.000000,552.000000,313.000000,313.000000,557.000000,433.000000,125.000000
hit_rate,0.982097,0.937833,0.980462,0.555950,0.555950,0.989343,0.769094,0.222025
ppv,0.950495,0.758621,0.529750,0.874302,0.902017,0.743658,0.900208,0.475285
f1,0.966038,0.838761,0.687850,0.679696,0.687912,0.849085,0.829502,0.302663
RTO,-0.005208,1.267045,0.407609,2.958466,2.958466,-0.775583,-4.866051,-2.576000
RTD,1.421551,1.539472,1.646960,3.661587,3.661587,1.426190,3.366123,4.420433


#### Annotator MN

In [25]:
mn_onset_alignments = alignment_metrics.xs(
    peyes.constants.ONSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('MN', level=u.GT_STR).dropna(axis=1, how='all')
mn_onset_alignments = mn_onset_alignments.reindex(
    columns=u.sort_labelers(mn_onset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

mn_onset_alignments

pred,RA,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
num_pred,391.000000,502.000000,729.000000,257.000000,247.000000,539.000000,327.000000,171.000000
num_matches,375.000000,380.000000,395.000000,175.000000,174.000000,400.000000,293.000000,116.000000
hit_rate,0.928218,0.940594,0.977723,0.433168,0.430693,0.990099,0.725248,0.287129
ppv,0.959079,0.756972,0.541838,0.680934,0.704453,0.742115,0.896024,0.678363
f1,0.943396,0.838852,0.697264,0.529501,0.534562,0.848356,0.801642,0.403478
RTO,0.917333,-3.776316,-0.126582,-12.902857,-13.040230,-1.315000,-0.177474,2.077586
RTD,4.023411,4.779888,4.277013,6.345009,6.098199,4.068264,5.795953,4.191845


In [26]:
mn_offset_alignments = alignment_metrics.xs(
    peyes.constants.OFFSET_STR, level=peyes.constants.CHANNEL_TYPE_STR, axis=0
).stack(u.GT_STR, future_stack=True).xs('MN', level=u.GT_STR).dropna(axis=1, how='all')
mn_offset_alignments = mn_offset_alignments.reindex(
    columns=u.sort_labelers(mn_offset_alignments.columns.get_level_values(u.PRED_STR).unique()), level=1
)

mn_offset_alignments

pred,RA,ivt,ivvt,idt,idvt,engbert,nh,remodnav
metric,,,,,,,,
num_gt,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000,404.000000
num_pred,391.000000,502.000000,729.000000,257.000000,247.000000,539.000000,327.000000,171.000000
num_matches,384.000000,385.000000,397.000000,226.000000,226.000000,401.000000,307.000000,77.000000
hit_rate,0.950495,0.952970,0.982673,0.559406,0.559406,0.992574,0.759901,0.190594
ppv,0.982097,0.766932,0.544582,0.879377,0.914980,0.743970,0.938838,0.450292
f1,0.966038,0.849890,0.700794,0.683812,0.694316,0.850477,0.839945,0.267826
RTO,0.005208,1.241558,0.342569,3.013274,3.013274,-0.698254,-4.960912,-1.987013
RTD,1.421551,1.368035,1.639723,3.185254,3.185254,1.463171,3.751886,4.075564


## Statistical Analysis

In [27]:
statistics, pvalues, dunns, Ns = ctd.kruskal_wallis_dunns(thresholded_time_diffs, [GT1, GT2], multi_comp=MULTI_COMP)
statistics.index.names = pvalues.index.names = dunns.index.names = [peyes.constants.CHANNEL_TYPE_STR]

pvalues <= ALPHA

gt,MN,RA
channel_type,,
offset,True,True
onset,True,True


In [28]:
pd.concat([statistics, pvalues], axis=1, keys=['H', 'p']).stack(1, future_stack=True)

H              p
channel_type gt                            
offset       MN  1311.200108  4.074620e-280
             RA  1932.148168   0.000000e+00
onset        MN   742.128974  4.885965e-157
             RA   980.160995  1.745909e-208

#### Post Hoc Analysis

In [29]:
post_hoc_onset = ctd.post_hoc_table(dunns, peyes.constants.ONSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_onset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --     ***     ***     ***     ***     ***      ***
         RA      --     ***     ***     ***     ***     ***      ***
ivvt     MN  0.0000      --     ***     ***       *    n.s.      ***
         RA  0.0000      --     ***     ***       *    n.s.      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0171  0.0000  0.0000      --    n.s.      ***
         RA  0.0000  0.0224  0.0000  0.0000      --      **      ***
nh       MN  0.0000  1.0000  0.0000  0.0000  0.1378      --      ***
         RA  0.0000  1.0000  0.0000  0.0000  0.0034      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  0.0000  0.0000       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0000  0.0001       --

In [30]:
post_hoc_offset = ctd.post_hoc_table(dunns, peyes.constants.OFFSET_STR, [GT1, GT2], alpha=ALPHA, marginal_alpha=MARGINAL_ALPHA)
post_hoc_offset

pred            ivt    ivvt     idt    idvt engbert      nh remodnav
pred     gt                                                         
ivt      MN      --     ***     ***     ***     ***     ***      ***
         RA      --     ***     ***     ***     ***     ***      ***
ivvt     MN  0.0000      --     ***     ***     ***     ***      ***
         RA  0.0000      --     ***     ***     ***     ***      ***
idt      MN  0.0000  0.0000      --    n.s.     ***     ***      ***
         RA  0.0000  0.0000      --    n.s.     ***     ***      ***
idvt     MN  0.0000  0.0000  1.0000      --     ***     ***      ***
         RA  0.0000  0.0000  1.0000      --     ***     ***      ***
engbert  MN  0.0000  0.0000  0.0000  0.0000      --     ***     n.s.
         RA  0.0000  0.0000  0.0000  0.0000      --     ***        *
nh       MN  0.0000  0.0000  0.0000  0.0000  0.0000      --       **
         RA  0.0000  0.0000  0.0000  0.0000  0.0000      --      ***
remodnav MN  0.0000  0.0000  0.0000  0.0000  1.0000  0.0010       --
         RA  0.0000  0.0000  0.0000  0.0000  0.0267  0.0005       --

### RTO Distribution
#### (RTD is the s.t.d. around RTO) 

In [31]:
temporal_alignment_fig = ctd.distributions_figure(
    thresholded_time_diffs, GT1, gt2=GT2, only_box=False, show_other_gt=True, share_x=True,
    colors={k: v[1] for k, v in LABELER_PLOTTING_CONFIG.items()},
    subplots_vspace=0.15,
)
temporal_alignment_fig.update_traces(width=0.9)     # make violins wider so there's less space between them

temporal_alignment_fig.update_layout(
    title=None,
    width=W, height=H,
    paper_bgcolor='rgba(0, 0, 0, 0)', plot_bgcolor='rgba(0, 0, 0, 0)',
    yaxis=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    yaxis2=dict(range=[-MATCHING_THRESHOLD, MATCHING_THRESHOLD], showgrid=False, zeroline=True, title='Δt (samples)'),
    margin=dict(l=10, r=10, b=10, t=20, pad=0),
    showlegend=False,
)

# update subplot titles:
for annotation in temporal_alignment_fig.layout.annotations:
    annotation.text = f"{EVENT_TYPE} {annotation.text}"
# temporal_alignment_fig.layout.annotations = []    # remove subtitles

save_fig(temporal_alignment_fig, FIG_ID, PANEL_ID, 'temporal-alignment_fixation', IS_SUPP)
temporal_alignment_fig.show()